## Honour Code
I SIBUSISO, NGWENYA, confirm - by submitting my - that the solutions in this notebook are a result of my own work and that I abide by the EDSA honour code (https://drive.google.com/file/d/1QDCjGZJ8-FmJE3bZdIQNwnJyQKPhHZBn/view?usp=sharing).

Non-compliance with the honour code constitutes a material breach of contract.

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import pandas_profiling as pp

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold, cross_validate
from catboost import CatBoostClassifier
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing


import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['training.csv', 'test.csv']


In [2]:
train = pd.read_csv('../input/training.csv')
test = pd.read_csv('../input/test.csv')

In [3]:
display(train.head())
display(test.head())

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy,FraudResult
0,TransactionId_76871,BatchId_36123,AccountId_3957,SubscriptionId_887,CustomerId_4406,UGX,256,ProviderId_6,ProductId_10,airtime,ChannelId_3,1000.0,1000,2018-11-15T02:18:49Z,2,0
1,TransactionId_73770,BatchId_15642,AccountId_4841,SubscriptionId_3829,CustomerId_4406,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-20.0,20,2018-11-15T02:19:08Z,2,0
2,TransactionId_26203,BatchId_53941,AccountId_4229,SubscriptionId_222,CustomerId_4683,UGX,256,ProviderId_6,ProductId_1,airtime,ChannelId_3,500.0,500,2018-11-15T02:44:21Z,2,0
3,TransactionId_380,BatchId_102363,AccountId_648,SubscriptionId_2185,CustomerId_988,UGX,256,ProviderId_1,ProductId_21,utility_bill,ChannelId_3,20000.0,21800,2018-11-15T03:32:55Z,2,0
4,TransactionId_28195,BatchId_38780,AccountId_4841,SubscriptionId_3829,CustomerId_988,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-644.0,644,2018-11-15T03:34:21Z,2,0


,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy
0,TransactionId_50600,BatchId_35028,AccountId_2441,SubscriptionId_4426,CustomerId_2857,UGX,256,ProviderId_5,ProductId_3,airtime,ChannelId_3,1000.0,1000,2019-02-13T10:01:40Z,4
1,TransactionId_95109,BatchId_45139,AccountId_3439,SubscriptionId_2643,CustomerId_3874,UGX,256,ProviderId_5,ProductId_15,financial_services,ChannelId_3,2000.0,2000,2019-02-13T10:02:12Z,2
2,TransactionId_47357,BatchId_74887,AccountId_4841,SubscriptionId_3829,CustomerId_2857,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-50.0,50,2019-02-13T10:02:30Z,2
3,TransactionId_28185,BatchId_11025,AccountId_2685,SubscriptionId_4626,CustomerId_3105,UGX,256,ProviderId_5,ProductId_10,airtime,ChannelId_3,3000.0,3000,2019-02-13T10:02:38Z,4
4,TransactionId_22140,BatchId_29804,AccountId_4841,SubscriptionId_3829,CustomerId_3105,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-60.0,60,2019-02-13T10:02:58Z,2


In [4]:
# Checking for missing data.
print('Missing data in training set:', train.isnull().sum().sum())
print('Missing data in testing set:', test.isnull().sum().sum())

Missing data in training set: 0
Missing data in testing set: 0


In [5]:
train.FraudResult.unique()

array([0, 1])

In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95662 entries, 0 to 95661
Data columns (total 16 columns):
TransactionId           95662 non-null object
BatchId                 95662 non-null object
AccountId               95662 non-null object
SubscriptionId          95662 non-null object
CustomerId              95662 non-null object
CurrencyCode            95662 non-null object
CountryCode             95662 non-null int64
ProviderId              95662 non-null object
ProductId               95662 non-null object
ProductCategory         95662 non-null object
ChannelId               95662 non-null object
Amount                  95662 non-null float64
Value                   95662 non-null int64
TransactionStartTime    95662 non-null object
PricingStrategy         95662 non-null int64
FraudResult             95662 non-null int64
dtypes: float64(1), int64(4), object(11)
memory usage: 11.7+ MB


# Transforming To DateTime

In [7]:
train['TransactionStartTime'] =  pd.to_datetime(train['TransactionStartTime'])
test['TransactionStartTime'] =  pd.to_datetime(test['TransactionStartTime'])

# New Columns

In [8]:
train['Year'] = train['TransactionStartTime'].dt.year
train['Month'] = train['TransactionStartTime'].dt.month
train['Day'] = train['TransactionStartTime'].dt.day
train['Time'] = train['TransactionStartTime'].dt.time

In [9]:
test['Year'] = test['TransactionStartTime'].dt.year
test['Month'] = test['TransactionStartTime'].dt.month
test['Day'] = test['TransactionStartTime'].dt.day
test['Time'] = test['TransactionStartTime'].dt.time

# Int To Object

In [10]:
train['PricingStrategy'] = train['PricingStrategy'].astype(object)

In [11]:
test['PricingStrategy'] = test['PricingStrategy'].astype(object)

# Profile Report

In [12]:
pp.ProfileReport(train)

Number of variables,20
Number of observations,95662
Total Missing (%),0.0%
Total size in memory,14.6 MiB
Average record size in memory,160.0 B
Numeric,4
Categorical,9
Boolean,2
Date,1
Text (Unique),1
Rejected,3


# Deleting Columns

In [13]:
train = train.drop(['AccountId', 'BatchId', 'CountryCode', 'CurrencyCode', 'CustomerId', 'SubscriptionId', 
                    'TransactionStartTime', 'ProductId', 'ProductCategory', 'ChannelId', 'Time'], axis=1)

In [14]:
test = test.drop(['AccountId', 'BatchId', 'CountryCode', 'CurrencyCode', 'CustomerId', 'SubscriptionId', 
                  'TransactionStartTime', 'ProductId', 'ProductCategory', 'ChannelId', 'Time'], axis=1)

# Defining X and y

In [15]:
y = train.FraudResult
X_train = train.drop(['FraudResult', 'TransactionId'], axis=1)

In [16]:
X_test = test.drop('TransactionId', axis=1)

# Dummy Variables

In [17]:
X_train = pd.get_dummies(X_train, drop_first=True)
X_test = pd.get_dummies(X_test, drop_first=True)

# Normalising

In [18]:
# X_train = X_train.values 
# min_max_scaler = preprocessing.MinMaxScaler()

# X_train = min_max_scaler.fit_transform(X_train)

In [19]:
# X_test = X_test.values 
# min_max_scaler = preprocessing.MinMaxScaler()

# X_test = min_max_scaler.fit_transform(X_test)

# Cross Validation 

In [20]:
# def cross_val(model, kfolds_number):
    
#     classifier = model
#     kfolds = kfolds_number
#     score = {'acc': 'accuracy', 'f1': 'f1_micro'} 
    
#     val_results = cross_validate(classifier, X_train, y, scoring=score, cv=kfolds, n_jobs=-1, verbose=1)
    
#     for values in val_results: 
#         print(values + ' : ', val_results[values].mean())

# Logistic Regression

In [21]:
# cross_val(LogisticRegression(), 6)

# CatBoost

In [22]:
# cross_val(CatBoostClassifier(), 6)

# Fitting

In [23]:
cat = CatBoostClassifier()
model = cat.fit(X_train, y)

Learning rate set to 0.060908
0:	learn: 0.5011931	total: 112ms	remaining: 1m 52s
1:	learn: 0.3378736	total: 174ms	remaining: 1m 27s
2:	learn: 0.2228605	total: 237ms	remaining: 1m 18s
3:	learn: 0.1690054	total: 292ms	remaining: 1m 12s
4:	learn: 0.1079480	total: 352ms	remaining: 1m 10s
5:	learn: 0.0748691	total: 412ms	remaining: 1m 8s
6:	learn: 0.0494642	total: 471ms	remaining: 1m 6s
7:	learn: 0.0350408	total: 531ms	remaining: 1m 5s
8:	learn: 0.0242328	total: 591ms	remaining: 1m 5s
9:	learn: 0.0179359	total: 651ms	remaining: 1m 4s
10:	learn: 0.0136605	total: 710ms	remaining: 1m 3s
11:	learn: 0.0107824	total: 769ms	remaining: 1m 3s
12:	learn: 0.0088402	total: 833ms	remaining: 1m 3s
13:	learn: 0.0073621	total: 893ms	remaining: 1m 2s
14:	learn: 0.0061839	total: 955ms	remaining: 1m 2s
15:	learn: 0.0050368	total: 1.01s	remaining: 1m 2s
16:	learn: 0.0042880	total: 1.07s	remaining: 1m 2s
17:	learn: 0.0038187	total: 1.14s	remaining: 1m 2s
18:	learn: 0.0034650	total: 1.2s	remaining: 1m 1s
19:	lea

# Predictions

In [24]:
y_pred = model.predict(X_test)

# Submission

In [25]:
submit = pd.DataFrame({'TransactionId': test.TransactionId, 'FraudResult': y_pred})

In [26]:
submit.to_csv('submission_norm.csv', index=False)